In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# import sys
# sys.path.append('/content/drive/MyDrive/Major_Project_Cse')

%cd "/content/drive/MyDrive/Major_Project_Cse"

Mounted at /content/drive
/content/drive/MyDrive/Major_Project_Cse


In [ ]:
import pandas as pd

# read the data from the CSV file
data = pd.read_csv('train_data.csv')

# load the keypoints and labels into separate variables
# train_keypoints = list(data['keypoints'])
train_keypoints=[]
train_labels = list(data['label'])

In [ ]:
numOfKeypoints = 34
import ast
for keypoints_str in data['keypoints']:
    keypoints_list = ast.literal_eval(keypoints_str)
    train_keypoints.append(keypoints_list)

In [ ]:
import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import initializers
from keras.layers import Conv3D, MaxPooling3D,Dense,ZeroPadding3D,Flatten,Activation,BatchNormalization,RandomFlip,RandomRotation,Rescaling
from keras.models import load_model


In [ ]:
type(train_keypoints[0][0][1][0])

NoneType

In [ ]:
# calculate the center of the object in each frame
centers = []
for keypoints in train_keypoints:
    frame_centers = []
    for frame_keypoints in keypoints:
        # assume the center is the mean of all keypoints
       center_x = np.nanmean([kp[0] for kp in frame_keypoints if kp[0] is not None])
       center_y = np.nanmean([kp[1] for kp in frame_keypoints if kp[1] is not None])
       frame_centers.append((center_x, center_y))
    centers.append(frame_centers)

train_relative_keypoints = train_keypoints
# make the keypoints relative to the center
for i in range(len(train_relative_keypoints)):
    for j in range(len(train_relative_keypoints[i])):
        for k in range(len(train_relative_keypoints[i][j])):
            if train_relative_keypoints[i][j][k][0] is not None:
               train_relative_keypoints[i][j][k][0] -= centers[i][j][0]
            if train_relative_keypoints[i][j][k][1] is not None:
              train_relative_keypoints[i][j][k][1] -= centers[i][j][1]


<ipython-input-6-a6c48355f099>:7: RuntimeWarning: Mean of empty slice
  center_x = np.nanmean([kp[0] for kp in frame_keypoints if kp[0] is not None])
<ipython-input-6-a6c48355f099>:8: RuntimeWarning: Mean of empty slice
  center_y = np.nanmean([kp[1] for kp in frame_keypoints if kp[1] is not None])


In [ ]:
count=0
new_train_keypoints = []
new_train_labels = []
for i in range(len(train_relative_keypoints)):
    flag=1
    frames_keypoints=[]
    for j in range(len(train_relative_keypoints[i])):
        for k in range(len(train_relative_keypoints[i][j])):
            if train_relative_keypoints[i][j][k][0] is None or train_relative_keypoints[i][j][k][1] is None:
              count+=1
              flag=0
              break
        if(flag==1):
          frames_keypoints.append(train_relative_keypoints[i][j])
    if(len(frames_keypoints)>0):
      kk = [[0.0,0.0] for _ in range(numOfKeypoints)]
      for i in range (16-len(frames_keypoints)):
          frames_keypoints.append(kk)
      # if(len(frames_keypoints)!=16):
      #   print("2 : ",len(frames_keypoints))
      new_train_keypoints.append(frames_keypoints)
      new_train_labels.append(train_labels[i])

In [ ]:
print(len(new_train_keypoints),len(new_train_labels))

984 984


In [ ]:
# train_videos = np.asarray(train_videos)
new_train_keypoints = np.asarray(new_train_keypoints)
new_train_labels = np.asarray(new_train_labels).astype('int32')

# train_keypoints = tf.convert_to_tensor(train_keypoints, dtype=tf.float32)
# # yKey_train = tf.cast(yKey_train , dtype=tf.float32)
# train_labels = tf.convert_to_tensor(train_labels, dtype=tf.float32)

# train_keypoints = np.array([np.array(val) for val in train_keypoints])

from sklearn.model_selection import train_test_split

XKey_train,XKey_test,yKey_train,yKey_test = train_test_split(new_train_keypoints,new_train_labels,test_size=0.25,shuffle=True)

In [ ]:
print(len(XKey_train),len(XKey_test))

738 246


In [ ]:
XKey_train = tf.convert_to_tensor(XKey_train, dtype=tf.float32)
yKey_train = tf.convert_to_tensor(yKey_train, dtype=tf.int32)

In [ ]:
from keras import models, layers
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, LSTM, Dense, Dropout, TimeDistributed, GlobalAveragePooling1D
from keras.optimizers import Adam
from keras.initializers import GlorotNormal
from keras.regularizers import L1L2

yKey_train = np.squeeze(yKey_train)

tf.config.run_functions_eagerly(True)

# Define model input shape
input_shape = (16, 34, 2)

# Build model architecture
model = models.Sequential()

model.add(TimeDistributed(Conv1D(filters=16, kernel_size=3, activation='relu'),input_shape = input_shape))
# model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Conv1D(filters=128, kernel_size=3, activation='relu')))
# model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Conv1D(filters=50, kernel_size=3, activation='relu')))
# model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))

# flatten output from CNN layers
model.add(TimeDistributed(GlobalAveragePooling1D()))

# add LSTM layer
model.add(LSTM(100, dropout=0.0, recurrent_dropout=0.0, kernel_initializer=GlorotNormal(seed=42), recurrent_regularizer=L1L2(l1=0.0, l2=0.001), bias_regularizer=L1L2(l1=0.0, l2=0.001), recurrent_initializer=GlorotNormal(seed=42), return_sequences=False))
# model.add(LSTM(100, recurrent_regularizer=L1L2(l1=0.01, l2=0.001), bias_regularizer=L1L2(l1=0.01, l2=0.001), recurrent_initializer=GlorotNormal(seed=42), return_sequences=False))

# TODO: forget bias 
# TODO: L1 = 0.01


# add fully connected layers

# model.add(Dropout(0.5))
# model.add(Dense(100, activation='relu'))

model.add(Dropout(0.5))
model.add(Dense(80, activation='relu'))

# add Softmax layer
model.add(Dense(20, activation='softmax'))

# compile model
adam = Adam(learning_rate=0.0001) 
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

# print model summary
# print(model.summary())

model.fit(XKey_train,yKey_train,batch_size=32,epochs = 300,validation_split = 0.2)

Epoch 1/300


/usr/local/lib/python3.9/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


19/19 [==============================] - 5s 250ms/step - loss: 3.2490 - accuracy: 0.0797 - val_loss: 3.2444 - val_accuracy: 0.0541
Epoch 2/300
 3/19 [===>..........................] - ETA: 2s - loss: 3.2420 - accuracy: 0.0729

KeyboardInterrupt: ignored

In [ ]:
# import mediapipe as mp

# landmarks = mp_pose.PoseLandmark.__members__
# print(landmarks)


{'NOSE': <PoseLandmark.NOSE: 0>, 'LEFT_EYE_INNER': <PoseLandmark.LEFT_EYE_INNER: 1>, 'LEFT_EYE': <PoseLandmark.LEFT_EYE: 2>, 'LEFT_EYE_OUTER': <PoseLandmark.LEFT_EYE_OUTER: 3>, 'RIGHT_EYE_INNER': <PoseLandmark.RIGHT_EYE_INNER: 4>, 'RIGHT_EYE': <PoseLandmark.RIGHT_EYE: 5>, 'RIGHT_EYE_OUTER': <PoseLandmark.RIGHT_EYE_OUTER: 6>, 'LEFT_EAR': <PoseLandmark.LEFT_EAR: 7>, 'RIGHT_EAR': <PoseLandmark.RIGHT_EAR: 8>, 'MOUTH_LEFT': <PoseLandmark.MOUTH_LEFT: 9>, 'MOUTH_RIGHT': <PoseLandmark.MOUTH_RIGHT: 10>, 'LEFT_SHOULDER': <PoseLandmark.LEFT_SHOULDER: 11>, 'RIGHT_SHOULDER': <PoseLandmark.RIGHT_SHOULDER: 12>, 'LEFT_ELBOW': <PoseLandmark.LEFT_ELBOW: 13>, 'RIGHT_ELBOW': <PoseLandmark.RIGHT_ELBOW: 14>, 'LEFT_WRIST': <PoseLandmark.LEFT_WRIST: 15>, 'RIGHT_WRIST': <PoseLandmark.RIGHT_WRIST: 16>, 'LEFT_PINKY': <PoseLandmark.LEFT_PINKY: 17>, 'RIGHT_PINKY': <PoseLandmark.RIGHT_PINKY: 18>, 'LEFT_INDEX': <PoseLandmark.LEFT_INDEX: 19>, 'RIGHT_INDEX': <PoseLandmark.RIGHT_INDEX: 20>, 'LEFT_THUMB': <PoseLandmark.

In [ ]:
from keras import models, layers
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, LSTM, Dense, Dropout, TimeDistributed, GlobalAveragePooling1D
from keras.optimizers import Adam
from keras.initializers import GlorotNormal
from keras.regularizers import L1L2

yKey_train = np.squeeze(yKey_train)

tf.config.run_functions_eagerly(True)

# Define model input shape
input_shape = (16, 34, 2)

# Build model architecture
model = models.Sequential()

model.add(TimeDistributed(Conv1D(filters=16, kernel_size=3, activation='relu'),input_shape = input_shape))
# model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Conv1D(filters=128, kernel_size=3, activation='relu')))
# model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Conv1D(filters=50, kernel_size=3, activation='relu')))
# model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))

# flatten output from CNN layers
model.add(TimeDistributed(GlobalAveragePooling1D()))

# add LSTM layer
# model.add(LSTM(100, dropout=0.0, recurrent_dropout=0.0, kernel_initializer=GlorotNormal(seed=42), recurrent_regularizer=L1L2(l1=0.0, l2=0.001), bias_regularizer=L1L2(l1=0.0, l2=0.001), recurrent_initializer=GlorotNormal(seed=42), return_sequences=False))
model.add(LSTM(100, recurrent_regularizer=L1L2(l1=0.01, l2=0.001), bias_regularizer=L1L2(l1=0.01, l2=0.001), recurrent_initializer=GlorotNormal(seed=42), return_sequences=False))

# TODO: forget bias 
# TODO: L1 = 0.01


# add fully connected layers

# model.add(Dropout(0.5))
# model.add(Dense(100, activation='relu'))

model.add(Dropout(0.5))
model.add(Dense(80, activation='relu'))

# add Softmax layer
model.add(Dense(20, activation='softmax'))

# compile model
adam = Adam(learning_rate=0.001) 
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

# print model summary
# print(model.summary())

model.fit(XKey_train,yKey_train,batch_size=32,epochs = 300,validation_split = 0.2)

Epoch 1/300
19/19 [==============================] - 4s 217ms/step - loss: 21.8459 - accuracy: 0.0593 - val_loss: 18.2877 - val_accuracy: 0.1284
Epoch 2/300
19/19 [==============================] - 4s 236ms/step - loss: 15.7570 - accuracy: 0.0864 - val_loss: 12.9646 - val_accuracy: 0.1757
Epoch 3/300
19/19 [==============================] - 5s 255ms/step - loss: 11.0284 - accuracy: 0.1390 - val_loss: 8.7999 - val_accuracy: 0.1622
Epoch 4/300
19/19 [==============================] - 4s 209ms/step - loss: 7.4553 - accuracy: 0.2356 - val_loss: 5.9011 - val_accuracy: 0.3243
Epoch 5/300
19/19 [==============================] - 4s 195ms/step - loss: 5.1357 - accuracy: 0.3203 - val_loss: 4.1856 - val_accuracy: 0.4054
Epoch 6/300
19/19 [==============================] - 5s 285ms/step - loss: 3.7443 - accuracy: 0.3644 - val_loss: 3.2042 - val_accuracy: 0.4730
Epoch 7/300
19/19 [==============================] - 4s 191ms/step - loss: 3.0305 - accuracy: 0.3831 - val_loss: 2.7632 - val_accuracy: 0

In [ ]:
from keras import models, layers
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, LSTM, Dense, Dropout, TimeDistributed, GlobalAveragePooling1D
from keras.optimizers import Adam
from keras.initializers import GlorotNormal
from keras.regularizers import L1L2

yKey_train = np.squeeze(yKey_train)

tf.config.run_functions_eagerly(True)

# Define model input shape
input_shape = (16, 34, 2)

# Build model architecture
model = models.Sequential()

model.add(TimeDistributed(Conv1D(filters=16, kernel_size=3, activation='relu'),input_shape = input_shape))
# model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Conv1D(filters=128, kernel_size=3, activation='relu')))
# model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Conv1D(filters=50, kernel_size=3, activation='relu')))
# model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))

# flatten output from CNN layers
model.add(TimeDistributed(GlobalAveragePooling1D()))

# add LSTM layer
model.add(LSTM(100, dropout=0.0, recurrent_dropout=0.0, kernel_initializer=GlorotNormal(seed=42), recurrent_regularizer=L1L2(l1=0.0, l2=0.001), bias_regularizer=L1L2(l1=0.0, l2=0.001), recurrent_initializer=GlorotNormal(seed=42), return_sequences=False))
# model.add(LSTM(100, recurrent_regularizer=L1L2(l1=0.01, l2=0.001), bias_regularizer=L1L2(l1=0.01, l2=0.001), recurrent_initializer=GlorotNormal(seed=42), return_sequences=False))

# TODO: forget bias 
# TODO: L1 = 0.01


# add fully connected layers

# model.add(Dropout(0.5))
# model.add(Dense(100, activation='relu'))

model.add(Dropout(0.5))
model.add(Dense(80, activation='relu'))

# add Softmax layer
model.add(Dense(20, activation='softmax'))

# compile model
adam = Adam(learning_rate=0.0001) 
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

# print model summary
# print(model.summary())

model.fit(XKey_train,yKey_train,batch_size=32,epochs = 500,validation_split = 0.2)

Epoch 1/500
19/19 [==============================] - 8s 428ms/step - loss: 3.2529 - accuracy: 0.1051 - val_loss: 3.2474 - val_accuracy: 0.1554
Epoch 2/500
19/19 [==============================] - 10s 502ms/step - loss: 3.2432 - accuracy: 0.1237 - val_loss: 3.2367 - val_accuracy: 0.1486
Epoch 3/500
19/19 [==============================] - 6s 316ms/step - loss: 3.2331 - accuracy: 0.1475 - val_loss: 3.2263 - val_accuracy: 0.1959
Epoch 4/500
19/19 [==============================] - 5s 275ms/step - loss: 3.2234 - accuracy: 0.1492 - val_loss: 3.2156 - val_accuracy: 0.1757
Epoch 5/500
19/19 [==============================] - 4s 205ms/step - loss: 3.2123 - accuracy: 0.1492 - val_loss: 3.2026 - val_accuracy: 0.1419
Epoch 6/500
19/19 [==============================] - 4s 206ms/step - loss: 3.1988 - accuracy: 0.1661 - val_loss: 3.1848 - val_accuracy: 0.2095
Epoch 7/500
19/19 [==============================] - 5s 268ms/step - loss: 3.1795 - accuracy: 0.1593 - val_loss: 3.1553 - val_accuracy: 0.195

In [ ]:
from keras import models, layers
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, LSTM, Dense, Dropout, TimeDistributed, GlobalAveragePooling1D
from keras.optimizers import Adam
from keras.initializers import GlorotNormal
from keras.regularizers import L1L2

yKey_train = np.squeeze(yKey_train)

tf.config.run_functions_eagerly(True)

# Define model input shape
input_shape = (16, 34, 2)

# Build model architecture
model = models.Sequential()

model.add(TimeDistributed(Conv1D(filters=16, kernel_size=3, activation='relu'),input_shape = input_shape))
# model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Conv1D(filters=128, kernel_size=3, activation='relu')))
# model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Conv1D(filters=50, kernel_size=3, activation='relu')))
# model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))

# flatten output from CNN layers
model.add(TimeDistributed(GlobalAveragePooling1D()))

# add LSTM layer
# model.add(LSTM(100, dropout=0.0, recurrent_dropout=0.0, kernel_initializer=GlorotNormal(seed=42), recurrent_regularizer=L1L2(l1=0.0, l2=0.001), bias_regularizer=L1L2(l1=0.0, l2=0.001), recurrent_initializer=GlorotNormal(seed=42), return_sequences=False))
model.add(LSTM(100, recurrent_regularizer=L1L2(l1=0.01, l2=0.001), bias_regularizer=L1L2(l1=0.01, l2=0.001), recurrent_initializer=GlorotNormal(seed=42), return_sequences=False))

# TODO: forget bias 
# TODO: L1 = 0.01


# add fully connected layers

# model.add(Dropout(0.5))
# model.add(Dense(100, activation='relu'))

model.add(Dropout(0.5))
model.add(Dense(80, activation='relu'))

# add Softmax layer
model.add(Dense(20, activation='softmax'))

# compile model
adam = Adam(learning_rate=0.001) 
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

# print model summary
# print(model.summary())

history = model.fit(XKey_train,yKey_train,batch_size=32,epochs = 300,validation_split = 0.2)

Epoch 1/300
19/19 [==============================] - 5s 256ms/step - loss: 21.8499 - accuracy: 0.0593 - val_loss: 18.3173 - val_accuracy: 0.0608
Epoch 2/300
19/19 [==============================] - 3s 166ms/step - loss: 15.7733 - accuracy: 0.0949 - val_loss: 13.0139 - val_accuracy: 0.1486
Epoch 3/300
19/19 [==============================] - 4s 186ms/step - loss: 11.0386 - accuracy: 0.1339 - val_loss: 9.0068 - val_accuracy: 0.2297
Epoch 4/300
19/19 [==============================] - 4s 192ms/step - loss: 7.6009 - accuracy: 0.2305 - val_loss: 6.0978 - val_accuracy: 0.2905
Epoch 5/300
19/19 [==============================] - 4s 236ms/step - loss: 5.2643 - accuracy: 0.2915 - val_loss: 4.2287 - val_accuracy: 0.2973
Epoch 6/300
19/19 [==============================] - 3s 167ms/step - loss: 3.8139 - accuracy: 0.3441 - val_loss: 3.1436 - val_accuracy: 0.4392
Epoch 7/300
19/19 [==============================] - 3s 166ms/step - loss: 3.0444 - accuracy: 0.3627 - val_loss: 2.6204 - val_accuracy: 0

In [ ]:


# Get model summary
model.summary()

# # Get training and validation accuracy
# train_accuracy = history.history['accuracy']
# val_accuracy = history.history['val_accuracy']

# # Get training and validation loss
# train_loss = history.history['loss']
# val_loss = history.history['val_loss']


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_7 (TimeDis  (None, 16, 32, 16)       112       
 tributed)                                                       
                                                                 
 time_distributed_8 (TimeDis  (None, 16, 16, 16)       0         
 tributed)                                                       
                                                                 
 time_distributed_9 (TimeDis  (None, 16, 14, 128)      6272      
 tributed)                                                       
                                                                 
 time_distributed_10 (TimeDi  (None, 16, 7, 128)       0         
 stributed)                                                      
                                                                 
 time_distributed_11 (TimeDi  (None, 16, 5, 50)       

In [ ]:
model.evaluate(XKey_test,yKey_test)

/usr/local/lib/python3.9/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


8/8 [==============================] - 2s 169ms/step - loss: 0.5026 - accuracy: 0.9106


[0.5026221871376038, 0.9105691313743591]

In [ ]:
# Save the model to a file
# model.save('PoseStream.h5')

# Load the saved model from the file
loaded_model = load_model('PoseStream.h5')

In [ ]:
loaded_model.evaluate(XKey_test,yKey_test)

1/8 [==>...........................] - ETA: 0s - loss: 0.2957 - accuracy: 0.8750

/usr/local/lib/python3.9/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


8/8 [==============================] - 1s 119ms/step - loss: 0.5026 - accuracy: 0.9106


[0.5026221871376038, 0.9105691313743591]

In [ ]:
loaded_model.predict(new_train_keypoints[0])

/usr/local/lib/python3.9/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


ValueError: ignored

In [ ]:
input_data = new_train_keypoints[0]
# Reshape the input data to include the batch size dimension
input_data = np.expand_dims(input_data, axis=0)

# Make a prediction with the model
output = model.predict(input_data)
output

/usr/local/lib/python3.9/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


1/1 [==============================] - 0s 299ms/step


array([[9.99708712e-01, 2.62607158e-11, 8.06678546e-09, 1.93838652e-08,
        4.37154535e-09, 3.91757027e-07, 7.37984745e-19, 7.65237303e-07,
        2.77937128e-04, 2.42370291e-11, 1.20226105e-05, 3.20784697e-12,
        1.26892900e-07, 1.34451783e-09, 6.71775569e-13, 3.91274763e-10,
        7.01205912e-13, 3.00378666e-10, 9.27593319e-15, 4.94650870e-13]],
      dtype=float32)